Objectif :
    Le but de ce code est de détecter et afficher des gestes de la main en temps réel
      à partir d'un flux vidéo de webcam dans un notebook.

Voici une description plus détaillée :
  - Accès à la webcam et affichage du flux : Le code utilise la webcam de l'utilisateur
      pour capturer un flux vidéo en temps réel. Ce flux est ensuite affiché dans
      le notebook Colab grâce à une intégration HTML et JavaScript.
  - Détection des mains et des points de repère : Grâce à la bibliothèque MediaPipe Hands,
      le code identifie les mains présentes dans chaque image du flux vidéo et repère des points
      clés sur la main (articulations des doigts, poignet, etc.).
  - Reconnaissance de gestes : Le code inclut une fonction detect_gesture qui analyse la position
      des points de repère de la main pour reconnaître deux gestes spécifiques : le signe "OK"
      (pouce et index formant un cercle) et le signe "Salut" (main ouverte avec les doigts écartés).
  - Affichage des résultats : Les points de repère de la main sont dessinés sur l'image du flux vidéo
      pour une visualisation. Si un geste est reconnu, le nom du geste ("OK" ou "Salut") est affiché sur l'image.
  - Interaction en temps réel : Tout ce processus se déroule en temps réel, ce qui permet à l'utilisateur d'interagir
      avec le système en effectuant les gestes reconnus devant la webcam.


In [1]:
# 0. Preparation Environnement
# 0.1. Installer les packages requis
!pip install -r requirements.txt 
!pip install pandas matplotlib seaborn scikit-learn opencv-python mediapipe -q

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 0.2. Charger les librairies 
import cv2
import mediapipe as mp
import numpy as np

from IPython.display import display, HTML


In [3]:
# 1. Initialisation de MediaPipe Hands
"""
Note: 
    - 'mp_hands': pour récupérer le module de détection des mains dans MediaPipe
    - 'hands': pour initialiser le détecteur de mains avec un suil de confiance pour la détection et le suivi
    - 'mp_drawing': pour dessiner les points de repère (landmarks) sur l'image
"""

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [4]:
# 2. Fonction Détection de Gestes
"""
Note: 
    - 'detect_gesture': fonction qui prend en entrée les landmarks d'une main détectée et retourne le geste reconnu
        - OK: Si le bout du pouce est proche de l'index
        - Salut: Si tous les doigts (sauf le pouce) sont étendus et au dessus du poignet
    - 'thumb_tip', 'index_tip', 'pinky_tip': points de repère pour le bout du pouce, l'index et l'auriculaire
    - Conditions pour reconnaître les gestes "OK" et "Salut"
"""

def detect_gesture(hand_landmarks):
    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]

    # Signe OK : bout du pouce proche de l'index
    if abs(thumb_tip.x - index_tip.x) < 0.05 and abs(thumb_tip.y - index_tip.y) < 0.05:
        return "OK"

    # Signe de salut : main ouverte, doigts écartés
    fingers_extended = [l.y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
                        for l in [hand_landmarks.landmark[i]
                        for i in [mp_hands.HandLandmark.INDEX_FINGER_TIP,
                                  mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                                  mp_hands.HandLandmark.RING_FINGER_TIP,
                                  mp_hands.HandLandmark.PINKY_TIP]]]
    if all(fingers_extended):
        return "Salut"

    return None

In [5]:
# 3. Démarrer le flux vidéo avec miroir et détection de gestes
"""
Note: 
    - 'display(HTML(...))': pour afficher le flux vidéo dans le notebook => Spécifique à Colab
    - 'cv2.VideoCapture(0)": pour ouvrir la webcam
    - 'cv2.flip(frame, 1)': pour retourner l'image horizontalement (effet miroir)
    - 'hands.process(image)': pour détecter les mains dans l'image
    - 'mp_drawing.draw_landmarks(...)': pour dessiner les points de repère de la main sur l'image
    - 'detect_gesture(hand_landmarks)': pour détecter les gestes à partir des points de repère
"""

def start_video_stream_with_mirror_and_landmarks():
    # Supprimez la ligne display(HTML) car elle est spécifique à Colab
    capture = cv2.VideoCapture(0)
    
    # Créer une fenêtre OpenCV
    cv2.namedWindow('Hand Detection', cv2.WINDOW_NORMAL)

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            break
            
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        gesture = None
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = detect_gesture(hand_landmarks)

        if gesture:
            cv2.putText(frame, f"Detected Gesture : {gesture}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Remplacer cv2_imshow par cv2.imshow
        cv2.imshow('Hand Detection', frame)
        
        # Ajouter une condition de sortie plus claire
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:  # q ou ESC pour quitter
            break
            
    capture.release()
    cv2.destroyAllWindows()

In [6]:
# 4. Lancer le programme
start_video_stream_with_mirror_and_landmarks()

In [7]:
# NEXT STEPS
"""
Dans les prochaines étapes, je vais essayer d'introduire d'autres gestes et de les reconnaître.
"""

"\nDans les prochaines étapes, je vais essayer d'introduire d'autres gestes et de les reconnaître.\n"